# get predictions from final models

In [1]:
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses
from keras.models import Model
from keras import backend as K
import scipy
import datetime 
import pickle
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

K.tensorflow_backend._get_available_gpus()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

from matplotlib import pyplot as plt
%matplotlib inline

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
CONSENSUS=False

#if set to CONSENSUS, will only save the consensus embedding. if set to false, will save all nodes across all runs
# medoid_assignments = pd.read_csv("MEDOIDS_INFO.csv").sort_values("cluster")
dset = 'Blood_GSE63060'
dpath = "_regimp"
# dpath = "_intersection"

path_to_final_models = "MDAD_models/"

In [76]:
if dpath=="intersection":
    Xs = [np.loadtxt("processed%s/%s/GE_PCA_test.txt"%(dpath,dset))]
else:
    Xs = [np.loadtxt("processed%s/%s/GE_PCA.txt"%(dpath,dset))]


phens = ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]
MTL_phenotype_output_mapping = {"BRAAK":0, "CERAD":1, "PLAQUES":2, "TANGLES":3, "ABETA_IHC":4, "TAU_IHC":5}

MTL_LAYER_IDXes = [1, 3,\
                  {"BRAAK": 5, "CERAD":6, "PLAQUES": 7, "TANGLES":8, "ABETA_IHC":9, "TAU_IHC":10}, 
                  {"BRAAK": 17, "CERAD":18, "PLAQUES": 19, "TANGLES":20, "ABETA_IHC":21, "TAU_IHC":22},
                  {"BRAAK": 29, "CERAD":30, "PLAQUES": 31, "TANGLES":32, "ABETA_IHC":33, "TAU_IHC":34}]

# MTL

In [78]:
def ignorenans_mse(y_true, y_pred):
    bool_finite = tf.is_finite(y_true)
    return K.mean(K.square(tf.boolean_mask(y_pred, bool_finite) - tf.boolean_mask(y_true, bool_finite)), axis=-1)



def get_model_layers(model_file, num_layers):
    
    # note: need to define custom functions for model in order to load, but these don't actually get used
    model = keras.models.load_model(model_file, custom_objects={"ignorenans_mse": ignorenans_mse})
    
    # define new model that cuts off the last several layers
    newmodel = Model(inputs = model.input, outputs = model.layers[num_layers-1].output)
    
    # agian, need to specify these parameters, but they aren't used since we don't retrain the model
    opt = optimizers.adam()  
    newmodel.compile(optimizer=opt, loss= "mse")
    
    return newmodel

In [80]:
if CONSENSUS: 
    embedding = np.zeros([len(X), len(medoid_assignments)])
    runs_to_check = medoid_assignments["run"].unique()

else:
    runs_to_check = np.arange(100)
    

for rep in runs_to_check:
    print(rep)
    
    if dpath == "_intersection":
        path_to_model = "---need to save new models somewhere----"
    else:
        path_to_model = path_to_final_models + "%i/200.hdf5"%(i)

    for HIDDEN_LAYER in [1]:


        mod_to_layer = get_model_layers(path_to_model, MTL_LAYER_IDXes[HIDDEN_LAYER]+1)

        X_transformed = mod_to_layer.predict(X)
        
        if CONSENSUS:
            for _,row in medoid_assignments[medoid_assignments["run"]==rep].iterrows():
                print(row)
                embedding[:,row["cluster"]] = X_transformed[:,row["node_idx"]]
                
                print(rep, row["cluster"], row["node_idx"])
                
        else:
            savepath = "model_transformations%s/%s/MTL/%i/"%(dpath,dset,HIDDEN_LAYER)
            if not os.path.isdir(savepath):
                os.makedirs(savepath)
            np.savetxt("%s%s.txt"%(savepath, rep),X_transformed)

savepath = "model_transformations%s/%s/MTL/%i/"%(dpath,dset,HIDDEN_LAYER)
if not os.path.isdir(savepath):
    os.makedirs(savepath)
savepath = "model_transformations%s/%s/MTL/%i/final.txt"%(dpath,dset,HIDDEN_LAYER)


0


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1


KeyboardInterrupt: 